<a href="https://colab.research.google.com/github/Volviane/AMMI_Assignments/blob/master/MFOGO_Volviane_Saphir_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chit Chat Chatbots

In the previous lab, we explored models that try to answer questions by reasoning over free-text input. In this lab, we will explore two types of models to create chatbots.

First, let's consider important qualities for a chit-chat chatbot system


1.   **Readability** - whatever model we use, the chats it creates should be easily understood by humans
2.   **Consistency** - when chatting with a chatbot, the bot should maintain consistent information. Imagine a bot that says "Hi I'm Jack'' and then "Hello, my name is Jane" - quite confusing
3.    **Engaging** - To encourage users to talk to the bot, the bot should be able to generate interesting, engaging responses. If the only response was "wow, that's cool," users are quite unlikely to want to talk very much to the chat bot



In [0]:
%%html
<p style='color: blue;'>
  Throughout the lab, there will be <b>questions</b> you should answer. <b>All questions you need to write an answer to will be in this blue color.</b>
  
  <br>Please write brief answers- no need for long explanations. 
  <br>There can be multiple correct answers to the questions.
  
  <br><br>The goal of these questions is to:
  <ul style='color: green;'>
    <li> Review the lecture material in the context of practical models and develop intuition about the models
    <li> Develop a sense of experimentation - we will pretend we have a dataset and will walk through an experimental thought process.
  </ul>

<b>We are going to do the lab as a group. <br>I will explain the sections in more depth, as we did not cover dialogue deeply during the lecture. <br> After we discuss, I will provide time for you to write a few sentences. At the end of the lab, you will hand it in. In theory, everyone should be finished together!</b>

  
</p>

## Data

The dataset we will use for this lab is called `PersonaChat` - it was created to directly address problem 2. Each person talking in the dataset has a personality, which helps maintain consistency in the dialogue. We saw it last week in the tutorials as well (when you worked through beam search)

In [0]:
!git clone https://github.com/facebookresearch/ParlAI.git ~/ParlAI  > /dev/null
!cd ~/ParlAI; python setup.py develop > /dev/null

Cloning into '/root/ParlAI'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 29939 (delta 5), reused 4 (delta 2), pack-reused 29911
Receiving objects: 100% (29939/29939), 57.59 MiB | 30.18 MiB/s, done.
Resolving deltas: 100% (21308/21308), done.
zip_safe flag not set; analyzing archive contents...
/usr/local/lib/python3.6/dist-packages/setuptools/dist.py:454: UserWarning: Normalizing '2019.08.19' to '2019.8.19'
  warnings.warn(tmpl.format(**locals()))
regex_3/_regex.c: In function ‘folded_char_at’:
regex_3/_regex.c:10625:9: warning: variable ‘folded_len’ set but not used [-Wunused-but-set-variable]
     int folded_len;
         ^~~~~~~~~~
regex_3/_regex.c: In function ‘fuzzy_match_group_fld’:
regex_3/_regex.c:11503:10: warning: ‘data.new_text_pos’ may be used uninitialized in this function [-Wmaybe-uninitialized]
     if (!record_fuzzy(state, data.fuzzy_type, data.new_text_pos - data.s

**Example: **

your persona: i just started college.

your persona: i have 3 science classes.

your persona: i work part time in the campus library.

your persona: i am living at home but hope to live in the dorms next year.

**Partner Dialogue**: hi how are you doing

**Your Response**: great ! just got off work and relaxing before i study

In [0]:
# let's download and take a look at some examples of data in PersonaChat
!python ~/ParlAI/examples/display_data.py --task personachat --datatype train

[creating task(s): personachat]
[building data: /root/ParlAI/data/Persona-Chat]
[ downloading: http://parl.ai/downloads/personachat/personachat.tgz to /root/ParlAI/data/Persona-Chat/personachat.tgz ]
[ Checksum Successful ]
unpacking personachat.tgz
[loading fbdialog data:/root/ParlAI/data/Persona-Chat/personachat/train_self_original.txt]
- - - NEW EPISODE: personachat - - -
your persona: i like to remodel homes.
your persona: i like to go hunting.
your persona: i like to shoot a bow.
your persona: my favorite holiday is halloween.
hi , how are you doing ? i am getting ready to do some cheetah chasing to stay in shape .
   you must be very fast . hunting is one of my favorite hobbies .
i am ! for my hobby i like to do canning or some whittling .
   i also remodel homes when i am not out bow hunting .
that is neat . when i was in high school i placed 6th in 100m dash !
   that is awesome . do you have a favorite season or time of year ?
i do not . but i do have a favorite meat since tha

In [1]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>What do the personalities look like?</li>
    <li>How does creating bots with these simple personalities address consistency for chatbots? </li>
    <li>What are some drawbacks/limitations of these specific personalities for addressing the problem of consistency?</li>
  </ul>
</p>

<p style='color: blue;'>
  <b>Answers:</b>
  <ul style='color: blue;'>
    <li>The fisrt personality look like a student who is working in part time.
        The second personality look like someone who works as a dental assistant during the day and who like party at night club. 
        These personalities are simple, natural and descriptive</li>
    <li>Since these personalities contain typical topics of human interest the bot should maintain consistent information for chatbots  just by using natural informations provide by the personalities. </li>
    <li>One of the limitations is generalization. The bot will not be able to say anything consistently if it not about these personalities.
          If you want to talk about a domain that is not represented in these personalities, you won't be able to do so because it is not in the personality.
         Due to the simplicity of these personalities, they are strong at providing as training data
for the beginning of conversations, when the two speakers do not know each other, focusing on asking and answering questions but when the conversations become much intense the bot  tend to loose  consistency.</li>
  </ul>
</p>



---


***Let's understand how much data we have. Let's compute the following using ParlAI:***


1.   **How many turns of data do we have?** In dialogue datasets, "amount of data" is measured in dialogue turns. Each time there is a single line of dialogue, that is called a "turn"
2.   **On average, how many words form a model input?**


---




In [0]:
!python ~/ParlAI/parlai/scripts/data_stats.py -t personachat -dt train -ltim 10000

[ note: changing datatype from train to train:ordered ]
[creating task(s): personachat]
[loading fbdialog data:/root/ParlAI/data/Persona-Chat/personachat/train_self_original.txt]
[ loaded 8939 episodes with a total of 65719 examples ]
23s elapsed:
      %done  \
    100.00%   
      exs  \
    65719   
                                                                                                                                                                                                                                                                                                                                                                                                                                                                 stats  \
    
input:
   utterances: 65719
   avg utterance length: 18.356974390967604
   tokens: 1206402
   unique tokens: 14209
   unique utterances: 64580
labels:
   utterances: 65719
   avg utterance length: 11.929411585690591
   tokens: 783989

## Evaluation

How are dialogue models evaluated?



1.   **Automatic Evaluation**: Hits @ 1, Hits @ 5, Hits @ 10, F1
2.   **Human Evaluation**: Pairing Selection, Human Rating



In [0]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>Take some notes about what these metrics are and what they mean here</li>
  </ul>
</p>

<p style='color: blue;'>
  <b>Answers:</b>
  <ul style='color: blue;'>
    
    <ul>
        <li>Automatic Evaluation means that we can run a script and get the answer imediatly.
            Where as for human evalution we basically need a human to actually read and decide how the model is doing.
        </li>

        <li>Automatic Evaluation is less time (money) consumming because we don't need to pay a human being to spend time and evaluate the medel.</li>
         <li> The Hits @ is very useful for retrieval base dialogue models.
             The Hits @ metric means that if you have a set of basic retrieval candidats then we will score all of these candidats for retrieval and the model retrieves the highest rank n for Hits @ n (n = 1, 5, 10 ) 
             
             The downside of Hits @ is that it can't be use to evaluate generative model since the generative model is restrictive that is why we use F1 metric for generative model</li>        
    </ul>

    <ul>
        <li>Human Evaluation:  pairing selection show people two things and ask them to choose the better model</li>
         <li> Human Evaluation : Human rating this usually happen on one to five scale, like you show someone what the model has predicted and you ask people to rate the prediction fo the model.</li>        
    </ul>

    
  </ul>
</p>

## Models

There are two main kinds of dialogue models. 

*Retrieval* Models analyze the current dialogue context and try to find appropriate responses in the dataset.

*Generative* Models analyze the current dialogue context
and try to write an answer, word by word, from left to right.
This can be thought of as an application of sequence-to-sequence models,  where the "encoder side" is the dialogue history and the "decoder side" is the dialogue response your chatbot should generate.

In [2]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>Let's discuss the pros/cons of retrieval compared to generative models - Are there settings when you might want to use one over the other?</li>
    <li>Compare a chit-chat application to something like booking a movie ticket- would you want to use generative, retrieval, or something else to accomplish that task? Why?</li>
    <li>How can you evaluate generative models with the metrics we discussed before? How do you think they will perform compared to retrieval models?</li>
    <li>In lecture, Antoine mentioned issues with generative model generation being generic and short. How does this happen in beam search?</li>
</ul>

<b> If you would like me to discuss how to actually use generative models in dialog, please say something! Otherwise, we will skip.</b>
</p>


<p style='color: blue;'>
  <b>Answers:</b>
  <ul style='color: blue;'>
    <li>In terms of response archive, retrieval models depends on response repository while gnerative models generate response from scratch
        In terms of grammatical mistakes, retrieval models don't make grammatical mistakes where as generative models do
        In terms of new cases, retrieval models can't handle unsee and new cases if pre-defined response do not exist while generative models is a smart system, design to handleunseen and new cases
        In terms of contextual information, retrieval models can't handle contextual entity information while the generative model can do that up to an extend
        Retrieval models are easy to handle in general but generative models are hard to handle 
    </li>
    <li>I would use  retrieval to accomplish that task because in this specific cas we want to push people througth  specific flot to make sure that the chatbot has correctly accomplish the task and it ables to give people the movie ticket they want. 
        In booking a movie ticket chat, all conversations will actually be very similar so retrieval models can be good. But I think I have to be careful with retrieval because when we will go further in the conversation, the answers will become more and more specific to the user and maybe the retrieval model won't work as well. At this step, we could continue the coversation using a generative model.
        We could for instance go with rule-based methods where we have a certainty that our system won't go outside the limits you designed.</li>
    <li>We can use F1 metric to evaluate generative models. How it will perform compared to retrieval models really depends on the dataset</li>
    <li>In beam search, the model predict m (size of the beam) probable next candidate. The beam search work by maximizing the probability over the sentence. So if we want to say something very interesting it always penalise by the probability on the training data that may have been not seen very often.</li>
</ul>


</p>

### Retrieval Models

Let's train a model to do retrieval first. We will try the *Memory Net.* 

In [0]:
# We can train a model with the following command: 
!python ~/ParlAI/examples/train_model.py -m kv_memnn -t personachat -dt train -veps 0.25 --model-file persona_chat_retrieval_model -vmt accuracy

# but we have limited time in the tutorial, so let's use an already pretrained model

Traceback (most recent call last):
  File "/root/ParlAI/examples/train_model.py", line 16, in <module>
    opt = parser.parse_args()
  File "/root/ParlAI/parlai/core/params.py", line 990, in parse_args
    self.add_extra_args(args)
  File "/root/ParlAI/parlai/core/params.py", line 830, in add_extra_args
    self.add_model_subargs(model)
  File "/root/ParlAI/parlai/core/params.py", line 738, in add_model_subargs
    agent = load_agent_module(model)
  File "/root/ParlAI/parlai/core/loader.py", line 133, in load_agent_module
    my_module = importlib.import_module(module_name)
  File "/usr/lib/python3.6/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 994, in _gcd_import
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen importlib._bootstrap>", line 941, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 219, in _call_with_fra

Quick Parameter Refresher:


*  `-m ` means which model we're going to use. Recall retrieval models are trained to rank the true response higher over a set of potential responses from the dataset (in ParlAI, these are called the "label candidates"). When it's time to write a dialogue response, the retrieval model returns the response that is ranked the highest
*  -`t` refers to the task. Here, we are training on PersonaChat data.
* `-dt` refers to the data split. We want to train our model, so we are using the training set.
* `-veps` refers to how often we should evaluate during training, our performance on validation. recall this is important because models, particularly neural ones, have the capacity to memorize the training dataset. So it's important to check how the model is doing on the validation set.
* `--model-file` refers to when your model is saved, what should the filename be
*  `-vmt` refers to the metric which we'll use to decide which model is the best. We'll cover this in the next section





**Let's interact with the model to get a sense of what it's learning. **How is this chat going to work?



1.   You will be assigned a persona. You will chat to the model by typing in the chat box.
2.   The chatbot also has a persona. It's secret and hidden from you!
3.   When you've finished chatting with this bot, type [DONE] and a new model persona will be assigned to the bot, so you can talk to a new bot. 
4.   When you move on to the next chatbot persona, the previous persona will be revealed. 

Interact with the chatbots and the personas. **Try to think about the following:**

*   Do the chatbots follow their persona a lot?
*   Was it difficult to follow your persona?





In [0]:
!python ~/ParlAI/projects/convai2/interactive.py -mf models:convai2/kvmemnn/model

[building data: /root/ParlAI/data/models/convai2/kvmemnn/kvmemnn.tgz]
[ downloading: http://parl.ai/downloads/_models/convai2/kvmemnn.tgz to /root/ParlAI/data/models/convai2/kvmemnn/kvmemnn.tgz ]
  File "/root/ParlAI/projects/convai2/interactive.py", line 127, in <module>
    interactive(parser.parse_args(print_args=False), print_parser=parser)
  File "/root/ParlAI/parlai/core/params.py", line 990, in parse_args
    self.add_extra_args(args)
  File "/root/ParlAI/parlai/core/params.py", line 828, in add_extra_args
    model = get_model_name(parsed)
  File "/root/ParlAI/parlai/core/params.py", line 122, in get_model_name
    model_file = modelzoo_path(opt.get('datapath'), model_file)
  File "/root/ParlAI/parlai/core/build_data.py", line 423, in modelzoo_path
    my_module.download(datapath)
  File "/root/ParlAI/parlai/zoo/convai2/kvmemnn.py", line 19, in download
    download_models(opt, fnames, 'convai2')
  File "/root/ParlAI/parlai/core/build_data.py", line 386, in download_models
    

In [3]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>What does this model seem to be doing well? What is it doing poorly? </li>
    <li>Why might it be performing poorly? What kind of experiment could you design to test your hypothesis?</li>
    <li>How do we know if we need to use a more complex model? Would we always want to use a more complex model? Why or why not?</li>
  </ul>
</p>

<p style='color: blue;'>
  <b>Answers:</b>
  <ul style='color: blue;'>
    <li>This model seem to be doing well sometimes in the begining of the coversation but when things start getting serious it start doing poorly.</li>
    <li>It performing poorly because the provided dataset (personality) is too small and sometimes is not decribe enought to make a long and consistence conversation. The experiment I could  design to test my hypothesis is just by chating with bot try to tell him anything that is not related to me and the bot will keep telling non consistence things
        instead of just telling me somethings like [I DON'T KNOW]
    </li>
    <li>We need to use a more complex model first of all when the task we want to solve is complex; and when the basline model we have is doing very poorly on the task, having in mind that the problem maybe the data and not the model.
        No we don't always need complex model. Because they are some tasks that we want to push people througth specific flot. So in this case we might want to use a simple rule-based model.
    </li>
  </ul>
</p>

In [0]:
# Here is a command to train a Transformer Ranker model if you would like to try it out
!python ~/ParlAI/examples/train_model.py -m transformer/ranker -t personachat -dt train -veps 0.25 --model-file persona_chat_retrieval_model -vmt accuracy

Streaming output truncated to the last 5000 lines.
    53.77   
    gpu_mem  \
     .02351   
         lr  \
   6.25e-06   
    mean_loss  \
        2.382   
    total_train_updates  \
                 311181   
     tpb  \
   11.66   
    updates  
        306

[ time:11677.0s total_exs:311488 epochs:4.74 ]
    clip  \
   .9837   
    exs  \
    307   
    gnorm  \
    53.25   
    gpu_mem  \
     .02351   
         lr  \
   6.25e-06   
    mean_loss  \
        2.631   
    total_train_updates  \
                 311488   
     tpb  \
   11.81   
    updates  
        307

[ time:11687.0s total_exs:311797 epochs:4.74 ]
    clip  \
   .9773   
    exs  \
    309   
    gnorm  \
    55.12   
    gpu_mem  \
     .02351   
         lr  \
   6.25e-06   
    mean_loss  \
        2.489   
    total_train_updates  \
                 311797   
     tpb  \
   12.14   
    updates  
        309

[ time:11697.0s total_exs:312105 epochs:4.75 ]
    clip  \
   .9773   
    exs  \
    308   
    gnor



---


An important aspect of training models is analyzing them. ***Try to answer the following questions.*** 

---




In [5]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>Are the models using the persona that we have provided? How can you tell? If I asked you to prove it to me, what experiments could you conduct? </li>
    <li>Previously, we computed some statistics about how long the persona is in the training data. The model has also only seen words present in the training dataset. But what happens if you push the model outside of what data it's been trained on? What kind of performance do you get? Why does this happen, and what could you do if you wanted to improve the model's ability to generalize? </li>
    <li>In ParlAI, we've set the parameters to save the model's best performance based on validation accuracy. What would happen if we saved the model based on the best training accuracy? Why does this happen? (if you like, try this out on your own and see the effect when you interact with the bot)</li>
  </ul>

<b> If you would like me to discuss how to use BERT in dialog, please say something! Otherwise, we will skip.</b>
</p>

<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>Yes the model is using the persona that we have provided.
        If I want to prove to you that the model is using the persona that we have provided, I would just write something out of the domain of my personnality and we will see that it will still use word comming from my personna.
        </li>
    <li>If you push the model outside of what data it's been trained on, it will not predict well,
        and we will get a poor performance. Because the model does not have the capabability to generalize on unseen data.
        We could train it on a very large dataset or use a generative model instead </li>
    <li>When evaluating a model, the trainning accuracy does not ccount that much we care about the generalization (test) accuracy.</li>
  </ul>
</p>

### [for self exploration] Generative Models

Generative models must produce word for word what they are going to say next in the dialogue. When predicting the next word, it produces a probability distribution over the entire vocabulary space for which word to generate next. To reduce the vocabulary space, we will use **byte-pair encoding** (BPE). 

*How does BPE work?* The BPE algorithm takes as input the training data and the number of *operations* it can do. It passes over the training set and tries to create sub-word units. For example, the word "beautiful" might be split into "beau" "ti" "ful". Each time it splits a word into sub-words, that is one operation. The final vocabulary output consists of these subwords. So "ful" can be part of "beautiful" and part of "fruitful" and so on.


**Questions to ask yourself**:


1.   Why is it important to keep the vocabulary space small?
2.   What does perplexity measure? Why would we use it as a training objective? 




In [0]:
# !python ~/ParlAI/examples/train_model.py -m transformer/generator -t personachat -dt train -veps 0.25 --model-file persona_chat_generative_model -vmt ppl

## Final Thoughts

**What did we learn about dialogue modeling? Review Questions to ask yourself**

*   How do retrieval models work? What about generative? What are their pros and cons?
*   What are some important traits of dialogue systems? How might the traits differ for different dialogue tasks?


**General Takeaways about Machine Learning and Experimentation:**

*   We don't try models just to try them - try to have a reason for conducting an experiment. As we did in the lab, try to analyze what's working well in your models and working poorly. Try to use these reasons to guide why you might want to try other models. Complex is not necessarily better. 
*   Certain models can be better for certain tasks. As we've seen, generative models are working really well for tasks such as machine translation, but have a bit to go before becoming general purpose dialogue generators. 



**I'm really interested in dialogue! What can I do to learn more?**


*   Play around in ParlAI: ParlAI is a general library with many great dialogue models and code for them. It also provides a standard interface to access datasets and interact with various models. 
*   Read the PersonaChat Paper: https://arxiv.org/pdf/1801.07243.pdf
*   Dialog using knowledge: One challenge of these chit chat systems is they do not concretely know any facts. So if you want to chat about a specific topic, the models cannot produce any relevant information - they say generic utterances or incorrect facts. One way to remedy this is to incorporate **knowledge** into the dialogue agents. This has been investigated in many different ways, but one of the first papers to show this is https://arxiv.org/abs/1811.01241. In this work, data is collected by asking one speaker to reference Wikipedia sentences.
* Dialog with BERT: pretty new,  there is an investigation of two ways to use BERT in this paper: https://arxiv.org/abs/1903.03094. 


